## QuSpin implementation

The [Lanczos decomposition](http://weinbe58.github.io/QuSpin/generated/quspin.tools.lanczos.expm_lanczos.html) $(E,V,Q)$ with initial state $v_0$ of a hermitian matrix $A$ can be used to compute the matrix exponential $\exp(aA)\ket{v_0}$ applied to the quantum state $\ket{v_0}$, without actually computing the exact matrix exponential.<br>
Let $A \approx Q T Q^\dagger$ with $T=V \mathrm{diag}(E) V^T$. Then, we can compute an approximation to the matrix exponential, applied to a state $\ket{\psi}$ as follows:
\begin{equation*}
\exp(a A)\ket{v_0} \approx Q \exp(a T) Q^\dagger \ket{v_0} = Q V \mathrm{diag}(e^{a E}) V^T Q^\dagger \ket{v_0}.
\end{equation*}
If we use $\ket{v_0}$ as the (nondegenerate) initial state for the Lanczos algorithm, then $\sum_{j,k}V^T_{ij}Q^\dagger_{jk}v_{0,k} = \sum_{j}V_{ji}\delta_{0,j} = V_{i,0}$ since by construction, $\ket{v_0}$ is the $0$-th row of $Q$ and all the rows are orthonormal, and the expression simplifies further.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font',**{'size':11, 'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)
rc('mathtext', **{'fontset':'cm'})
rc('xtick', labelsize = 11)
rc('ytick', labelsize = 11)
import numpy as np
from numpy import array, ndarray

from quspin.operators import hamiltonian, quantum_LinearOperator
from quspin.basis import boson_basis_1d
from quspin.tools.lanczos import lanczos_iter, expm_lanczos

In [2]:
def input_parameters(Nt: int, T: int) -> int:
    '''
    defines input parameters:
        - Nt: nr of time steps & nr of time-evolving unitary operators
        - N: nr of elements in the wave functions
        - T: time window in units of pi
        - controls are N (Nt) for the trapezoidal (rectangular) rule
    '''
    N = Nt + 1
    delta_t = T / Nt

    print('# time steps:\t', Nt)
    print('time step:\t', delta_t)

    return N, delta_t

In [3]:
N, delta_t = input_parameters(Nt=100, T=2.83)

N_trap = N
N_rect = N - 1

# time steps:	 100
time step:	 0.028300000000000002


In [4]:
dim = 3                 # global dimension
d = 3                   # local dimension
D_H = d**dim            # Hilbert space dimension
subdim = 2**dim         # subspace dimension for the gate transfer

In [5]:
# Hamiltonian parameters (in mega units): how to re-scale them properly?

twopi = 2 * np.pi
omega_1 = 5. * 1e3 * twopi
omega_2 = 5.5 * 1e3 * twopi
delta_1 = -350 * twopi
delta_2 = delta_1
omega_r = 7.5 * 1e3 * twopi
g_1 = 100 * twopi
g_2 = g_1

Delta_1 = omega_1 - omega_r
Delta_2 = omega_2 - omega_r
omega_tilde_1 = omega_1 + (g_1**2 / Delta_1)
omega_tilde_2 = omega_2 + (g_2**2 / Delta_2)

J = (g_1 * g_2 * (Delta_1 + Delta_2)) / (Delta_1 * Delta_2)
Delta = omega_tilde_2 - omega_tilde_1

In [6]:
# simple repeated parameters

Deltas = np.repeat(Delta, dim - 1)
deltas = np.repeat(delta_1, dim)
Js = np.repeat(J, dim)

rcontrol_start = np.random.uniform(-1, 1, N_trap)

# here we assume constant control for each transmon
u_n = 1 # rcontrol_start[0]

$$\hat{H}_n = \sum^{N}_{j=1}\left[\Delta_j\hat{b}^\dagger_j\hat{b}_j+\frac{1}{2}\delta_j\hat{b}^\dagger_j\hat{b}_j(\hat{b}^\dagger_j\hat{b}_j-1)+\sum_{\langle i, j\rangle}J_{ij}(\hat{b}^\dagger_i\hat{b}_j+\hat{b}_i\hat{b}^\dagger_j)+u^{(j)}_n(\hat{b}^\dagger_j+\hat{b}_j)\right]=\hat{H}_d+\hat{H}^c_n$$

In [7]:
# construct basis

basis = boson_basis_1d(L=dim, sps=d)

# define OBC site-coupling lists for operators

duff1 = [[Deltas[n], n] for n in range(dim-1)]
duff2 = [[1/2 * deltas[n], n, n] for n in range(dim)]

coup = [[Js[n], n, n+1] for n in range(dim-1)]
ctrl = [[u_n, n] for n in range(dim-1)]

In [8]:
# control Hamiltonian

stat = [['+', ctrl], ['-', ctrl]]
dyna = []

Hc = hamiltonian(stat, dyna, static_fmt='csr', dtype=np.float64, basis=basis, check_symm=False)

Hermiticity check passed!


In [9]:
# diagonalize control Hamiltonian

e, R = Hc.eigh()
rotate = lambda x: R.T @ x @ R
Hc = rotate(Hc.toarray()).diagonal()

In [10]:
# drift Hamiltonian

stat = [['n', duff1], ['nz', duff2], ['+-', coup], ['-+', coup]]

Hd = hamiltonian(stat, [], static_fmt='csr', dtype=np.float64, basis=basis, check_symm=False, check_herm=False)
# Hd = quantum_LinearOperator(stat, basis=basis, dtype=np.float64, check_symm=False, check_herm=False)

In [11]:
from quspin.operators import isquantum_LinearOperator

isquantum_LinearOperator(Hd)

False

quantum_LinearOperator [issue](https://github.com/weinbe58/QuSpin/issues/398) with "out" argument on GitHub.

In [12]:
# the initial state v0 used is the state the matrix exponential is evaluated on

np.random.seed(0)
v0 = np.random.rand(D_H)
E, V, Q_T = lanczos_iter(Hd, v0, 20)
prefactor = -1j * np.pi * delta_t
lanczos = expm_lanczos(E, V, Q_T, a = prefactor)

lanczos

array([ 0.12915596-1.01715318e-01j, -0.04100485-2.09507940e-01j,
       -0.08228383-1.67086283e-01j, -0.00644838-1.45385331e-01j,
       -0.11672361-1.50994402e-01j, -0.1540955 -6.52905572e-02j,
       -0.10047776-1.10502896e-01j, -0.31546015+1.05548372e-01j,
       -0.31116288+1.16433557e-01j,  0.04250193-1.31130772e-01j,
       -0.19306238-1.02257554e-01j, -0.08077983-4.44924217e-02j,
       -0.09931504+5.58366250e-02j, -0.04691773+2.81531783e-01j,
       -0.02032209+9.74185952e-03j, -0.22491383-7.21786709e-02j,
       -0.14528777+8.94553545e-02j, -0.21069431+8.89425074e-02j,
       -0.20407326-1.01968900e-01j, -0.21436374+8.67517177e-02j,
       -0.17840289+1.00896523e-01j, -0.00661893-8.69983992e-02j,
       -0.01989103-1.93625498e-02j, -0.2173712 +1.00039139e-01j,
        0.02528698+2.83366057e-02j,  0.1991274 +1.59100688e-02j,
        0.0429378 +7.62426218e-07j])

In [13]:
from scipy.linalg import expm

exact = expm(prefactor * Hd.toarray()) @ v0

In [14]:
np.flip(exact) - lanczos

array([ 0.01419733+0.10171532j,  0.70578361+0.26191351j,
        0.16648488+0.26346554j, -0.712458  +0.51879541j,
        0.06597094-0.0335276j ,  0.13173842-0.22812248j,
       -0.51434328+0.70782781j, -0.39341376+0.26524127j,
       -0.37427801-0.46559304j, -0.75302236+0.45701819j,
       -0.27734933+0.31024567j, -0.67050186-0.19633187j,
        0.03249742-0.02138j   , -0.09070561+0.65468339j,
       -0.35759205+0.39477519j, -0.21761098-0.11440579j,
       -0.43436146-0.45670763j,  0.34994533-0.52054355j,
       -0.80135706+0.32524803j, -0.74421889+0.00674376j,
       -0.14810718-0.47578297j, -0.56574928-0.06050462j,
       -0.24336531-0.45575621j,  0.20081962-0.59105204j,
       -0.28982127-0.58339494j, -0.33587724-0.71568322j,
        0.38839565-0.33933498j])

In [15]:
# rotating the drift Hamiltonian involves the dense representation
# Hd.matmat(R)

Possibly 2 limitations:

1. High density of the sparse drift Hamiltonian after rotation.
2. Is it possible to rotate the drift Hamiltonian in the control-diagonal basis with the representation as linear operator?